### 1. Multiple disk clutch brake

In [1]:
import numpy as np

# Constants for the problem
pi = np.pi

# Constants
mu = 0.5
s = 1.5
Ms = 40  # Nm
Mf = 3  # Nm
n = 250  # rpm
p_max = 1e6  # Pa (1 MPa)
Iz = 55e-6  # kg mm^2 to kg m^2
T_max = 15  # s
F_max = 1000  # N
r_imin = 55e-3  # m
r_omax = 110e-3  # m
delta_r = 20e-3  # delta_r = 20 mm
doh = 0.01 #Assumption

# Thickness bounds
t_max = 3e-3  # Max thickness in m
t_min = 1.5e-3  # Min thickness in m
l_max = 30e-3  # Max length in m
Z_max = 10  # Max number of disks
v_srmax = 10  # Max relative speed m/s

# Define bounds based on the existing constants
bounds = np.array([
    [r_imin, r_omax],  # r_o
    [r_imin, r_omax],  # r_i
    [t_min, t_max],    # t
    [1, Z_max],        # Z
    [1e-3, 10]         # rho (add a reasonable range for rho)
])

# Objective function
def objective(x):
    r_o, r_i, t, Z, rho = x
    return pi * (r_o**2 - r_i**2) * t * (Z + 1) * rho

# Constraint functions with checks
def constraint1(x):
    r_o, r_i, _, _, _ = x
    return r_o - r_i - delta_r

def constraint2(x):
    _, _, t, Z, _ = x
    return l_max - (Z + 1) * (t + doh)

def constraint3(x):
    r_o, r_i, _, _, _ = x
    if r_o == r_i:
        return 0  # or a very small number to indicate failure
    p_rz = F_max / (pi * (r_o**2 - r_i**2))
    return p_max - p_rz

def constraint4(x):
    r_o, r_i, _, _, _ = x
    if r_o == r_i:
        return 0  # or handle it differently as needed
    v_sr = (2 * pi * n * (r_o**3 - r_i**3)) / (90 * (r_o**2 - r_i**2))
    p_rz = F_max / (pi * (r_o**2 - r_i**2))
    return p_max * v_srmax - p_rz * v_sr

def constraint5(x):
    r_o, r_i, _, _, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    v_sr = (2 * pi * n * (r_o**3 - r_i**3)) / (90 * (r_o**2 - r_i**2))
    return v_srmax - v_sr

def constraint6(x):
    r_o, r_i, _, Z, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    M_h = (2/3) * mu * F_max * Z * (r_o**3 - r_i**3) / (r_o**2 - r_i**2)
    T = (Iz * pi * n) / (30 * (M_h + Mf))
    return T_max - T

def constraint7(x):
    r_o, r_i, _, Z, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    M_h = (2/3) * mu * F_max * Z * (r_o**3 - r_i**3) / (r_o**2 - r_i**2)
    return 1.5 * Ms - M_h

def constraint8(x):
    r_o, r_i, _, Z, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    M_h = (2/3) * mu * F_max * Z * (r_o**3 - r_i**3) / (r_o**2 - r_i**2)
    T = (Iz * pi * n) / (30 * (M_h + Mf))
    return T  # T >= 0

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7, constraint8]
    return np.array([con(x) for con in constraints])

sunflower Optimization(SOA) using above Objective Function

In [2]:
import numpy as np
import pandas as pd

def hybrid_optimization(fitness_function, constraints, num_plants, bounds, MaxDays, m, P, λ):
    """
    Hybrid Optimization Algorithm for the given problem.

    Parameters:
        fitness_function (callable): Objective function to minimize.
        constraints (list): List of constraint functions.
        num_plants (int): Number of plants (population size).
        bounds (np.ndarray): Variable bounds as a 2D array [[min1, max1], [min2, max2], ...].
        MaxDays (int): Maximum number of iterations.
        m (float): Percentage of worst plants to replace.
        P (float): Heat constant.
        λ (float): Step size scaling factor.

    Returns:
        tuple: Best solution found, its fitness value, and fitness evolution.
    """
    num_dimensions = bounds.shape[0]
    Xmin, Xmax = bounds[:, 0], bounds[:, 1]
    loss_values = []

    # Parameters
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability
    eta = np.random.rand()  # Weightage

    # Initialize population
    population = np.random.uniform(Xmin, Xmax, (num_plants, num_dimensions))

    # Define penalized fitness function
    def penalized_fitness(individual):
        penalties = sum(max(0, -con(individual)) for con in constraints)
        return fitness_function(individual) + 1e6 * penalties

    # Evaluate initial fitness
    fitness = np.array([penalized_fitness(ind) for ind in population])
    fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

    # Find the best solution
    sun_index = np.argmin(fitness)
    sun = population[sun_index]
    best_fitness = fitness[sun_index]
    loss_values.append(best_fitness)

    # Main loop
    for day in range(MaxDays):
        # Calculate direction vectors
        distances = np.linalg.norm(sun - population, axis=1)
        distances[distances == 0] = np.finfo(float).eps
        direction_vectors = (sun - population) / distances[:, np.newaxis]

        # Heat calculation
        heat = P / (4 * np.pi * distances**2)

        # Update population
        for i in range(num_plants):
            r = np.random.rand()
            step_size = λ * np.random.rand() * np.linalg.norm(Xmax - Xmin) / (2 * num_plants)
            if r < p:
                population[i] += ((np.random.rand() ** 2) * sun - population[i]) * fragrances[i] * eta + \
                                  (1 - eta) * step_size * direction_vectors[i]
            else:
                j = np.random.randint(0, num_plants)
                k = np.random.randint(0, num_plants)
                population[i] += ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta + \
                                  (1 - eta) * step_size * direction_vectors[i]

        # Keep population within bounds
        population = np.clip(population, Xmin, Xmax)

        # Evaluate new fitness
        fitness = np.array([penalized_fitness(ind) for ind in population])
        fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

        # Replace worst plants
        b = int(m / 100.0 * num_plants)
        worst_indices = np.argsort(fitness)[-b:]
        best_indices = np.argsort(fitness)[:b]
        best_plants = population[best_indices]

        for index, best_index in zip(worst_indices, best_indices):
            new_solution = (population[best_index] + population[best_index - 1]) / 2
            population[index] = np.clip(new_solution, Xmin, Xmax)

        # Update the best solution (sun)
        new_sun_index = np.argmin(fitness)
        if fitness[new_sun_index] < best_fitness:
            sun_index = new_sun_index
            sun = population[sun_index]
            best_fitness = fitness[new_sun_index]

        loss_values.append(best_fitness)
        a = np.random.rand()
        eta = np.random.rand()

        print(f"Day {day + 1}/{MaxDays}, Best fitness: {best_fitness}")

    return sun, best_fitness, loss_values


# Problem definition
problem = {
    "objective": objective,
    "constraints": [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7, constraint8],
    "bounds": bounds,
}

# Run the algorithm
num_plants = 30
MaxDays = 100
m = 20
P = 1.0
λ = 0.5

best_solution, best_fitness, fitness_evolution = hybrid_optimization(
    problem["objective"],
    problem["constraints"],
    num_plants,
    problem["bounds"],
    MaxDays,
    m,
    P,
    λ,
)

print("Best solution:", best_solution)
print("Best fitness:", best_fitness)

# Save results to Excel
output_file = "optimization_results.xlsx"
results = {
    "Day": list(range(1, len(fitness_evolution) + 1)),  # Dynamically match lengths
    "Fitness Evolution": fitness_evolution
}
results_df = pd.DataFrame(results)

# Save best solution and fitness
best_solution_df = pd.DataFrame({
    "Variable": [f"x{i+1}" for i in range(len(best_solution))],
    "Value": best_solution
})
best_solution_df.loc[len(best_solution_df)] = ["Best Fitness", best_fitness]

# Write to Excel file
with pd.ExcelWriter(output_file) as writer:
    results_df.to_excel(writer, index=False, sheet_name="Fitness Evolution")
    best_solution_df.to_excel(writer, index=False, sheet_name="Best Solution")

print(f"Results saved to {output_file}")



Day 1/100, Best fitness: 10832.43903293748
Day 2/100, Best fitness: 10832.43903293748
Day 3/100, Best fitness: 10832.43903293748
Day 4/100, Best fitness: 10832.43903293748
Day 5/100, Best fitness: 10832.43903293748
Day 6/100, Best fitness: 10832.43903293748
Day 7/100, Best fitness: 10832.43903293748
Day 8/100, Best fitness: 10832.43903293748
Day 9/100, Best fitness: 10832.43903293748
Day 10/100, Best fitness: 10832.43903293748
Day 11/100, Best fitness: 10832.43903293748
Day 12/100, Best fitness: 10832.43903293748
Day 13/100, Best fitness: 10832.43903293748
Day 14/100, Best fitness: 10832.43903293748
Day 15/100, Best fitness: 10832.43903293748
Day 16/100, Best fitness: 10832.43903293748
Day 17/100, Best fitness: 10832.43903293748
Day 18/100, Best fitness: 10832.43903293748
Day 19/100, Best fitness: 10832.43903293748
Day 20/100, Best fitness: 10832.43903293748
Day 21/100, Best fitness: 10832.43903293748
Day 22/100, Best fitness: 10832.43903293748
Day 23/100, Best fitness: 10832.439032937

### 2. Robot gripper

In [3]:
import numpy as np

# Constants for the problem
pi = np.pi
Y_min = 50
Y_max = 100
Y_G = 150
Z_max = 100
P = 100

# Variable bounds
a_min, a_max = 10, 150
b_min, b_max = 10, 150
f_min, f_max = 10, 150
c_min, c_max = 100, 200
e_min, e_max = 0, 50
l_min, l_max = 100, 300
doh_min, doh_max = 1, 3.14

# Define bounds based on the existing constants
bounds = np.array([
    [a_min, a_max],   # a
    [b_min, b_max],   # b
    [f_min, f_max],   # f
    [c_min, c_max],   # c
    [e_min, e_max],   # e
    [l_min, l_max],   # l
    [doh_min, doh_max]  # doh
])

# Helper functions for geometry calculations
def g(l, z, e):
    return np.sqrt((l - z)**2 + e**2) + 1e-10  # Small value to avoid division by zero

def alpha(a, b, l, z, e, phi):
    g_val = g(l, z, e)
    if g_val == 0 or a == 0:  # Avoid division by zero or invalid a
        return 0
    arg_alpha = (a**2 + g_val**2 - b**2) / (2 * a * g_val)
    arg_alpha_clipped = np.clip(arg_alpha, -1, 1)  # Ensure value is within [-1, 1]
    return np.arccos(arg_alpha_clipped) + phi

def beta(a, b, l, z, e, phi):
    g_val = g(l, z, e)
    if g_val == 0 or b == 0:  # Avoid division by zero or invalid b
        return 0
    arg_beta = (b**2 + g_val**2 - a**2) / (2 * b * g_val)
    arg_beta_clipped = np.clip(arg_beta, -1, 1)  # Ensure value is within [-1, 1]
    return np.arccos(arg_beta_clipped) - phi

def phi_func(e, l, z, phi_offset):
    return np.arctan(e / (l - z + 1e-10)) + phi_offset  # Avoid division by zero

# Redefined F_k function for minimization
def F_k_for_minimization(a, b, l, z, e, phi_offset, c):
    alpha_val = alpha(a, b, l, z, e, phi_offset)
    beta_val = beta(a, b, l, z, e, phi_offset)

    if np.cos(alpha_val) == 0:  # Avoid division by zero for cos(alpha)
        return 0

    return (P * b * np.sin(alpha_val + beta_val)) / (2 * c * np.cos(alpha_val))

# Objective function for minimization
def objective(x):
    a, b, f, c, e, l, doh = x
    z_min, z_max = 0, Z_max
    phi_offset = phi_func(e, l, z_min, doh)

    F_max = F_k_for_minimization(a, b, l, z_max, e, phi_offset, c)
    F_min = F_k_for_minimization(a, b, l, z_min, e, phi_offset, c)

    return F_max - F_min

# y(x, z) function
def y(x, z):
    a, b, f, c, e, l, doh = x
    phi = phi_func(e, l, z, doh)
    beta_val = beta(a, b, l, z, e, phi)
    return 2 * (e + f + c * np.sin(beta_val + doh))

# Constraint functions
def constraint1(x):
    return Y_min - y(x, Z_max)

def constraint2(x):
    return y(x, Z_max)

def constraint3(x):
    return y(x, 0) - Y_max

def constraint4(x):
    return Y_G - y(x, 0)

def constraint5(x):
    a, b, _, _, e, l, _ = x
    return (a + b)**2 - l**2 - e**2

def constraint6(x):
    a, b, _, _, e, l, _ = x
    return (l - Z_max)**2 + (a - e)**2 - b**2

def constraint7(x):
    _, _, _, _, _, l, _ = x
    return l - Z_max

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7]
    return np.array([con(x) for con in constraints])

Sunflower Optimization(SOA) using above Objective Function

In [5]:
import numpy as np
import pandas as pd

# Define the hybrid optimization algorithm
def hybrid_optimization(objective_func, constraints_func, bounds, num_plants, num_dimensions, MaxDays, m, P, λ):
    loss_values = []
    iteration_data = []  # To store iteration-wise data
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability
    eta = np.random.rand()  # Weightage

    # Step 1: Initialize the population of sunflowers randomly
    population = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_plants, num_dimensions))

    # Evaluate initial fitness and apply constraints
    fitness = np.apply_along_axis(objective_func, 1, population)
    constraints_violations = np.apply_along_axis(constraints_func, 1, population)

    # Check for constraints violation, and assign inf if violated
    violations = np.any(constraints_violations > 0, axis=1)
    fitness[violations] = np.inf

    # If all fitness values are inf, re-initialize the population to avoid deadlock
    if np.all(fitness == np.inf):
        print("All initial solutions violated constraints. Re-initializing population.")
        population = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_plants, num_dimensions))
        fitness = np.apply_along_axis(objective_func, 1, population)

    fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

    # Find the best solution (sun)
    sun_index = np.argmin(fitness)
    sun = population[sun_index]
    best_fitness = fitness[sun_index]
    loss_values.append(best_fitness)

    # Main loop (iteration over days)
    for day in range(MaxDays):
        distances = np.linalg.norm(sun - population, axis=1)
        distances[distances == 0] = np.finfo(float).eps
        direction_vectors = (sun - population) / distances[:, np.newaxis]

        heat = P / (4 * np.pi * distances**2)

        for i in range(len(population)):
            r = np.random.rand()
            step_size = λ * np.linalg.norm(population[i] - sun)
            if r < p:
                population[i] += (
                    ((np.random.rand() ** 2) * sun - population[i]) * fragrances[i] * eta
                    + (1 - eta) * step_size * direction_vectors[i]
                )
            else:
                j, k = np.random.randint(0, num_plants, 2)
                population[i] += (
                    ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta
                    + (1 - eta) * step_size * direction_vectors[i]
                )

        # Keep plants within bounds
        population = np.clip(population, bounds[:, 0], bounds[:, 1])

        # Evaluate new fitness and apply constraints
        fitness = np.apply_along_axis(objective_func, 1, population)
        constraints_violations = np.apply_along_axis(constraints_func, 1, population)

        # Check for constraints violation, and assign inf if violated
        violations = np.any(constraints_violations > 0, axis=1)
        fitness[violations] = np.inf

        # Update fragrances
        fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

        # Update the sun if a better solution is found
        new_sun_index = np.argmin(fitness)
        if fitness[new_sun_index] < best_fitness:
            sun_index = new_sun_index
            sun = population[sun_index]
            best_fitness = fitness[new_sun_index]
        loss_values.append(best_fitness)

        # Store the iteration data (fitness and loss values)
        iteration_data.append([day + 1, best_fitness])

        print(f"Iteration {day + 1}/{MaxDays}, Best fitness: {best_fitness}")

    return sun, best_fitness, loss_values, iteration_data


# Problem-specific setup
num_plants = 30
num_dimensions = bounds.shape[0]
MaxDays = 100
m = 20
P = 1.0
λ = 0.5

# Run the algorithm
best_solution, best_fitness, fitness_evolution, iteration_data = hybrid_optimization(
    objective_func=objective,
    constraints_func=evaluate_constraints,
    bounds=bounds,
    num_plants=num_plants,
    num_dimensions=num_dimensions,
    MaxDays=MaxDays,
    m=m,
    P=P,
    λ=λ
)

# Save the results in a dictionary
results = {
    "Best Solution": [best_solution],
    "Best Fitness": [best_fitness],
    "Fitness Evolution": [fitness_evolution]
}

# ... (previous code) ...

# Save results to CSV
output_file = "optimization_results.csv"  # Change file extension to .csv

# Create a DataFrame for fitness evolution
results_df = pd.DataFrame({
    "Day": list(range(1, len(fitness_evolution) + 1)),
    "Fitness Evolution": fitness_evolution
})

# Create a DataFrame for best solution and fitness
best_solution_df = pd.DataFrame({
    "Variable": [f"x{i+1}" for i in range(len(best_solution))],
    "Value": best_solution
})
best_solution_df.loc[len(best_solution_df)] = ["Best Fitness", best_fitness]

# Save to CSV files
results_df.to_csv(output_file, index=False)  # Save fitness evolution to CSV
best_solution_df.to_csv("best_solution.csv", index=False)  # Save best solution to a separate CSV

print(f"Fitness evolution saved to {output_file}")
print("Best solution saved to best_solution.csv")


All initial solutions violated constraints. Re-initializing population.
Iteration 1/100, Best fitness: -9.704041743918989
Iteration 2/100, Best fitness: -9.704041743918989
Iteration 3/100, Best fitness: -9.704041743918989
Iteration 4/100, Best fitness: -9.704041743918989
Iteration 5/100, Best fitness: -9.704041743918989
Iteration 6/100, Best fitness: -9.704041743918989
Iteration 7/100, Best fitness: -9.704041743918989
Iteration 8/100, Best fitness: -9.704041743918989
Iteration 9/100, Best fitness: -9.704041743918989
Iteration 10/100, Best fitness: -9.704041743918989
Iteration 11/100, Best fitness: -9.704041743918989


<ipython-input-5-d942eb718269>:57: RuntimeWarning: invalid value encountered in subtract
  ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta
<ipython-input-5-d942eb718269>:57: RuntimeWarning: invalid value encountered in multiply
  ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta


Iteration 12/100, Best fitness: -9.704041743918989
Iteration 13/100, Best fitness: -9.704041743918989
Iteration 14/100, Best fitness: -9.704041743918989
Iteration 15/100, Best fitness: -9.704041743918989
Iteration 16/100, Best fitness: -9.704041743918989
Iteration 17/100, Best fitness: -9.704041743918989
Iteration 18/100, Best fitness: -9.704041743918989
Iteration 19/100, Best fitness: -9.704041743918989
Iteration 20/100, Best fitness: -9.704041743918989
Iteration 21/100, Best fitness: -9.704041743918989
Iteration 22/100, Best fitness: -9.704041743918989
Iteration 23/100, Best fitness: -9.704041743918989
Iteration 24/100, Best fitness: -9.704041743918989
Iteration 25/100, Best fitness: -9.704041743918989
Iteration 26/100, Best fitness: -9.704041743918989
Iteration 27/100, Best fitness: -9.704041743918989
Iteration 28/100, Best fitness: -9.704041743918989
Iteration 29/100, Best fitness: -9.704041743918989
Iteration 30/100, Best fitness: -9.704041743918989
Iteration 31/100, Best fitness:

### 3. Step-cone pulley

In [6]:
import numpy as np

# Constants for the problem
pi = np.pi

# Constants in SI units
rho = 7200  # kg/m^3 (Density)
a = 3       # m (Length)
mu = 0.35   # (Dimensionless, Friction coefficient)
s = 1.75e6  # Pa (Stress, converted from 1.75 MPa)
t = 0.008 # m
N = 100     # (Total speed for normalization, units not specified)

# Objective function
def objective(x):
    if len(x) != 9:
        raise ValueError(f"The input vector must have exactly 9 elements, but received {len(x)}.")

    # Assign each value individually
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    # Limit values for numerical stability
    d1 = np.clip(d1, 1e-6, 1e3)
    d2 = np.clip(d2, 1e-6, 1e3)
    d3 = np.clip(d3, 1e-6, 1e3)
    d4 = np.clip(d4, 1e-6, 1e3)
    w = np.clip(w, 1e-6, 1e3)

    # Objective calculation
    term1 = d1**2 * (1 + (N1/N)**2)
    term2 = d2**2 * (1 + (N2/N)**2)
    term3 = d3**2 * (1 + (N3/N)**2)
    term4 = d4**2 * (1 + (N4/N)**2)

    # Handle overflow
    max_float = np.finfo(np.float64).max
    min_float = np.finfo(np.float64).min

    # Check for overflow and replace with max/min values
    term1 = term1 if term1 < max_float else max_float
    term2 = term2 if term2 < max_float else max_float
    term3 = term3 if term3 < max_float else max_float
    term4 = term4 if term4 < max_float else max_float

    # Check before final multiplication
    total_terms = term1 + term2 + term3 + term4
    if total_terms < max_float:
        result = rho * w * total_terms
    else:
        result = max_float  # If the total is still too high, set to max float

    return result

def constraint1(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    C1 = np.clip(np.pi * d1 / 2 * (1 + N1 / N) + ((N1/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)
    C2 = np.clip(np.pi * d2 / 2 * (1 + N2 / N) + ((N2/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)

    return C1 - C2

def constraint2(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    C1 = np.clip(np.pi * d1 / 2 * (1 + N1 / N) + ((N1/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)
    C3 = np.clip(np.pi * d3 / 2 * (1 + N3 / N) + ((N3/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)

    return C1 - C3

def constraint3(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    C1 = np.clip(np.pi * d1 / 2 * (1 + N1 / N) + ((N1/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)
    C4 = np.clip(np.pi * d4 / 2 * (1 + N4 / N) + ((N4/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)

    return C1 - C4

def constraint4(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    # Clip input to np.exp to avoid overflow
    R1 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N1/N - 1) * d1 / (2 * a), -1, 1))), -700, 700))
    R2 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N2/N - 1) * d2 / (2 * a), -1, 1))), -700, 700))
    R3 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N3/N - 1) * d3 / (2 * a), -1, 1))), -700, 700))
    R4 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N4/N - 1) * d4 / (2 * a), -1, 1))), -700, 700))

    return np.sum([R1, R2, R3, R4]) - 8

def constraint5(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    # Use np.clip to prevent overflow in np.exp
    P1 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N1/N - 1) * d1 / (2 * a), -1, 1))), -700, 700))) * (pi * d1 * N1 / 60)
    P2 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N2/N - 1) * d2 / (2 * a), -1, 1))), -700, 700))) * (pi * d2 * N2 / 60)
    P3 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N3/N - 1) * d3 / (2 * a), -1, 1))), -700, 700))) * (pi * d3 * N3 / 60)
    P4 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N4/N - 1) * d4 / (2 * a), -1, 1))), -700, 700))) * (pi * d4 * N4 / 60)

    return np.sum([P1, P2, P3, P4]) - (4 * 0.75 * 745.6998)

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5]
    return np.array([con(x) for con in constraints])

## Sunflower Optimization(**SOA**) using above Objective Function

In [8]:
import numpy as np
import pandas as pd

# Define the hybrid optimization algorithm
def hybrid_optimization(objective_func, constraints_func, bounds, num_plants, num_dimensions, MaxDays, m, P, λ):
    loss_values = []
    iteration_data = []  # To store iteration-wise data
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability
    eta = np.random.rand()  # Weightage

    # Step 1: Initialize the population of sunflowers randomly
    population = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_plants, num_dimensions))

    # Evaluate initial fitness and apply constraints
    fitness = np.apply_along_axis(objective_func, 1, population)
    constraints_violations = np.apply_along_axis(constraints_func, 1, population)

    # Check for constraints violation, and assign inf if violated
    violations = np.any(constraints_violations > 0, axis=1)
    fitness[violations] = np.inf

    # If all fitness values are inf, re-initialize the population to avoid deadlock
    if np.all(fitness == np.inf):
        print("All initial solutions violated constraints. Re-initializing population.")
        population = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_plants, num_dimensions))
        fitness = np.apply_along_axis(objective_func, 1, population)

    fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

    # Find the best solution (sun)
    sun_index = np.argmin(fitness)
    sun = population[sun_index]
    best_fitness = fitness[sun_index]
    loss_values.append(best_fitness)

    # Main loop (iteration over days)
    for day in range(MaxDays):
        distances = np.linalg.norm(sun - population, axis=1)
        distances[distances == 0] = np.finfo(float).eps
        direction_vectors = (sun - population) / distances[:, np.newaxis]

        heat = P / (4 * np.pi * distances**2)

        for i in range(len(population)):
            r = np.random.rand()
            step_size = λ * np.linalg.norm(population[i] - sun)
            if r < p:
                population[i] += (
                    ((np.random.rand() ** 2) * sun - population[i]) * fragrances[i] * eta
                    + (1 - eta) * step_size * direction_vectors[i]
                )
            else:
                j, k = np.random.randint(0, num_plants, 2)
                population[i] += (
                    ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta
                    + (1 - eta) * step_size * direction_vectors[i]
                )

        # Keep plants within bounds
        population = np.clip(population, bounds[:, 0], bounds[:, 1])

        # Evaluate new fitness and apply constraints
        fitness = np.apply_along_axis(objective_func, 1, population)
        constraints_violations = np.apply_along_axis(constraints_func, 1, population)

        # Check for constraints violation, and assign inf if violated
        violations = np.any(constraints_violations > 0, axis=1)
        fitness[violations] = np.inf

        # Update fragrances
        fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

        # Update the sun if a better solution is found
        new_sun_index = np.argmin(fitness)
        if fitness[new_sun_index] < best_fitness:
            sun_index = new_sun_index
            sun = population[sun_index]
            best_fitness = fitness[new_sun_index]
        loss_values.append(best_fitness)

        # Store the iteration data (fitness and loss values)
        iteration_data.append([day + 1, best_fitness])

        print(f"Iteration {day + 1}/{MaxDays}, Best fitness: {best_fitness}")

    return sun, best_fitness, loss_values, iteration_data


# Example Problem-specific setup
def objective(solution):
    # Define your objective function here
    return np.sum(solution**2)

def evaluate_constraints(solution):
    # Define constraint violations here (return 0 if no violation, else positive values)
    return np.zeros_like(solution)

# Define problem bounds
bounds = np.array([
    [1e-6, 1e3],  # Example bounds for a problem dimension
    [1e-6, 1e3]
])

# Run the algorithm
num_plants = 30
num_dimensions = bounds.shape[0]
MaxDays = 100
m = 20
P = 1.0
λ = 0.5

best_solution, best_fitness, fitness_evolution, iteration_data = hybrid_optimization(
    objective_func=objective,
    constraints_func=evaluate_constraints,
    bounds=bounds,
    num_plants=num_plants,
    num_dimensions=num_dimensions,
    MaxDays=MaxDays,
    m=m,
    P=P,
    λ=λ
)

# Display results
print("\n--- Final Results ---")
print(f"Best Solution: {best_solution}")
print(f"Best Fitness: {best_fitness}")
print(f"Fitness Evolution (last 10 iterations): {fitness_evolution[-10:]}")

# Save the results in a dictionary
results = {
    "Best Solution": [best_solution],
    "Best Fitness": [best_fitness],
    "Fitness Evolution": [fitness_evolution]
}

# ... (previous code) ...

# Save results to CSV
output_file = "optimization_results.csv"  # Change file extension to .csv

# Create a DataFrame for fitness evolution
results_df = pd.DataFrame({
    "Day": list(range(1, len(fitness_evolution) + 1)),
    "Fitness Evolution": fitness_evolution
})

# Create a DataFrame for best solution and fitness
best_solution_df = pd.DataFrame({
    "Variable": [f"x{i+1}" for i in range(len(best_solution))],
    "Value": best_solution
})
best_solution_df.loc[len(best_solution_df)] = ["Best Fitness", best_fitness]

# Save to CSV files
results_df.to_csv(output_file, index=False)  # Save fitness evolution to CSV
best_solution_df.to_csv("best_solution.csv", index=False)  # Save best solution to a separate CSV

print(f"Fitness evolution saved to {output_file}")
print("Best solution saved to best_solution.csv")

Iteration 1/100, Best fitness: 9914.018002491426
Iteration 2/100, Best fitness: 9367.458714551254
Iteration 3/100, Best fitness: 9099.685557054565
Iteration 4/100, Best fitness: 8860.808817030644
Iteration 5/100, Best fitness: 8589.35280544826
Iteration 6/100, Best fitness: 8445.628248365898
Iteration 7/100, Best fitness: 8207.762245799997
Iteration 8/100, Best fitness: 7996.476019750402
Iteration 9/100, Best fitness: 7794.389017662322
Iteration 10/100, Best fitness: 7528.046180272715
Iteration 11/100, Best fitness: 7331.500774539525
Iteration 12/100, Best fitness: 7182.870689216353
Iteration 13/100, Best fitness: 7002.845361533004
Iteration 14/100, Best fitness: 6732.136782178919
Iteration 15/100, Best fitness: 6548.991218108175
Iteration 16/100, Best fitness: 6380.9587698374935
Iteration 17/100, Best fitness: 6213.448150909903
Iteration 18/100, Best fitness: 6055.9037698936245
Iteration 19/100, Best fitness: 5851.515064754776
Iteration 20/100, Best fitness: 5655.281889054153
Iteratio

### 4. Hydrodynamic thrust bearing design

In [9]:
import numpy as np

# Constants for the problem
pi = np.pi

# Constants
gamma = 0.0307
C = 0.5
n = -3.55
C1 = 10.04
Ws = 101000  # N
Pmax = 1000  # Pa (1 MPa)
delta_Tmax = 50  # max delta temperature
h_min = 0.001  # min height in m
g = 386.4  # acceleration due to gravity in m/s^2
N = 750  # rpm

# Define variable bounds
R_min = 1  # Minimum radius
R_max = 16  # Maximum radius
Ro_min = 1  # Minimum inner radius
Ro_max = 16  # Maximum inner radius
Q_min = 1  # Minimum flow rate
Q_max = 16  # Maximum flow rate
mu_min = 1e-6  # Minimum viscosity
mu_max = 16e-6  # Maximum viscosity

# Define bounds based on the existing constants
bounds = np.array([
    [R_min, R_max],   # R
    [Ro_min, Ro_max], # R_o
    [Q_min, Q_max],   # Q
    [mu_min, mu_max]  # mu
])

# Objective function
def objective(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Add a small value to avoid division by zero

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Prevent log(0)

    # Calculate P and check for invalid log input
    log_input_for_P = 8.122e6 * mu + 0.8
    if log_input_for_P <= 0:
        P = 0  # Set P to 0 if the logarithm input is invalid
    else:
        log_input_for_P = max(log_input_for_P, 1e-10)  # Prevent log of zero
        P = (np.log10(np.log10(log_input_for_P)) - C1) / n

    delta_T = 2 * (10**P - 560)  # delta temperature
    Ef = 9336 * Q * gamma * delta_T
    return Po / 0.7 + Ef

# Height calculation based on other parameters
def calculate_h(R, Ro, Q, mu):
    log_input_for_P = 8.122e6 * mu + 0.8
    if log_input_for_P <= 0:
        P = 0  # Set P to 0 if the logarithm input is invalid
    else:
        log_input_for_P = max(log_input_for_P, 1e-10)  # Prevent log of zero
        P = (np.log10(np.log10(log_input_for_P)) - C1) / n

    delta_T = 2 * (10**P - 560)  # delta temperature
    Ef = 9336 * Q * gamma * (delta_T)  # Calculate E_f based on the new delta_T
    return (2 * pi * N / 60)**2 * (2 * pi * mu / Ef) * (R**4 / 4 - Ro**4 / 4)

# Constraint functions
def constraint1(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    if Po == 0:
        return np.inf  # Return a very high value for infeasibility

    if R <= Ro:  # Check if R is greater than Ro
        return np.inf  # Return a very high value for infeasibility

    W = np.pi * Po / 2 * (R**2 - Ro**2) / np.log(R / (Ro + 1e-9))
    return W - Ws

def constraint2(x):
    _, _, Q, mu = x
    h = calculate_h(*x) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = x[0] / (x[1] + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    return Pmax - Po

def constraint3(x):
    _, _, _, mu = x
    log_input_for_P = 8.122e6 * mu + 0.8
    if log_input_for_P <= 0:
        P = 0  # Set P to 0 if the logarithm input is invalid
    else:
        P = (np.log10(np.log10(log_input_for_P)) - C1) / n
    delta_T = 2 * (10**P - 560)  # delta temperature
    return delta_Tmax - delta_T

def constraint4(x):
    _, _, _, _ = x
    h = calculate_h(*x) + 1e-9  # Avoid zero height
    return h - h_min

def constraint5(x):
    R, Ro, _, _ = x
    return R - Ro

def constraint6(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    # Check for division by zero issues
    if Po == 0 or h == 0:  # Ensure Po and h are valid
        return np.inf  # Return a very high value for infeasibility

    return 0.001 - gamma / (g * Po) * (Q / (2 * np.pi * R * h))

def constraint7(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    if Po == 0:
        return np.inf  # Return a very high value for infeasibility

    if R <= Ro:  # Check if R is greater than Ro
        return np.inf  # Return a very high value for infeasibility

    W = np.pi * Po / 2 * (R**2 - Ro**2) / np.log(R / (Ro + 1e-9))
    return 5000 - W / (np.pi * (R**2 - Ro**2 + 1e-9))

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7]
    return np.array([con(x) for con in constraints])

# Sunflower Optimization(SOA) using above Objective Function

In [11]:
import numpy as np
import pandas as pd

# Hybrid Optimization Algorithm for the given problem
def hybrid_optimization(objective_func, constraints_func, bounds, num_plants, num_dimensions, MaxDays, m, P, λ):
    loss_values = []
    iteration_data = []  # To store iteration-wise data
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability
    eta = np.random.rand()  # Weightage

    # Step 1: Initialize the population of sunflowers randomly
    population = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_plants, num_dimensions))

    # Evaluate initial fitness and apply constraints
    fitness = np.apply_along_axis(objective_func, 1, population)
    constraints_violations = np.apply_along_axis(constraints_func, 1, population)

    # Check for constraints violation, and assign inf if violated
    violations = np.any(constraints_violations > 0, axis=1)
    fitness[violations] = np.inf

    # If all fitness values are inf, re-initialize the population to avoid deadlock
    if np.all(fitness == np.inf):
        print("All initial solutions violated constraints. Re-initializing population.")
        population = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_plants, num_dimensions))
        fitness = np.apply_along_axis(objective_func, 1, population)

    fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

    # Find the best solution (sun)
    sun_index = np.argmin(fitness)
    sun = population[sun_index]
    best_fitness = fitness[sun_index]
    loss_values.append(best_fitness)

    # Main loop (iteration over days)
    for day in range(MaxDays):
        distances = np.linalg.norm(sun - population, axis=1)
        distances[distances == 0] = np.finfo(float).eps
        direction_vectors = (sun - population) / distances[:, np.newaxis]

        heat = P / (4 * np.pi * distances**2)

        for i in range(len(population)):
            r = np.random.rand()
            step_size = λ * np.linalg.norm(population[i] - sun)
            if r < p:
                population[i] += (
                    ((np.random.rand() ** 2) * sun - population[i]) * fragrances[i] * eta
                    + (1 - eta) * step_size * direction_vectors[i]
                )
            else:
                j, k = np.random.randint(0, num_plants, 2)
                population[i] += (
                    ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta
                    + (1 - eta) * step_size * direction_vectors[i]
                )

        # Keep plants within bounds
        population = np.clip(population, bounds[:, 0], bounds[:, 1])

        # Evaluate new fitness and apply constraints
        fitness = np.apply_along_axis(objective_func, 1, population)
        constraints_violations = np.apply_along_axis(constraints_func, 1, population)

        # Check for constraints violation, and assign inf if violated
        violations = np.any(constraints_violations > 0, axis=1)
        fitness[violations] = np.inf

        # Update fragrances
        fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

        # Update the sun if a better solution is found
        new_sun_index = np.argmin(fitness)
        if fitness[new_sun_index] < best_fitness:
            sun_index = new_sun_index
            sun = population[sun_index]
            best_fitness = fitness[new_sun_index]
        loss_values.append(best_fitness)

        # Store the iteration data (fitness and loss values)
        iteration_data.append([day + 1, best_fitness])

        print(f"Iteration {day + 1}/{MaxDays}, Best fitness: {best_fitness}")

    return sun, best_fitness, loss_values, iteration_data


# Problem-specific setup
num_plants = 30
num_dimensions = bounds.shape[0]
MaxDays = 100
m = 20
P = 1.0
λ = 0.5

# Run the algorithm
best_solution, best_fitness, fitness_evolution, iteration_data = hybrid_optimization(
    objective_func=objective,
    constraints_func=evaluate_constraints,
    bounds=bounds,
    num_plants=num_plants,
    num_dimensions=num_dimensions,
    MaxDays=MaxDays,
    m=m,
    P=P,
    λ=λ
)

results = {
    "Best Solution": [best_solution],
    "Best Fitness": [best_fitness],
    "Fitness Evolution": [fitness_evolution]
}

# ... (previous code) ...

# Save results to CSV
output_file = "optimization_results.csv"  # Change file extension to .csv

# Create a DataFrame for fitness evolution
results_df = pd.DataFrame({
    "Day": list(range(1, len(fitness_evolution) + 1)),
    "Fitness Evolution": fitness_evolution
})

# Create a DataFrame for best solution and fitness
best_solution_df = pd.DataFrame({
    "Variable": [f"x{i+1}" for i in range(len(best_solution))],
    "Value": best_solution
})
best_solution_df.loc[len(best_solution_df)] = ["Best Fitness", best_fitness]

# Save to CSV files
results_df.to_csv(output_file, index=False)  # Save fitness evolution to CSV
best_solution_df.to_csv("best_solution.csv", index=False)  # Save best solution to a separate CSV

print(f"Fitness evolution saved to {output_file}")
print("Best solution saved to best_solution.csv")


All initial solutions violated constraints. Re-initializing population.
Iteration 1/100, Best fitness: -143471.31656365754
Iteration 2/100, Best fitness: -143471.31656365754
Iteration 3/100, Best fitness: -143471.31656365754
Iteration 4/100, Best fitness: -143471.31656365754
Iteration 5/100, Best fitness: -143471.31656365754
Iteration 6/100, Best fitness: -143471.31656365754
Iteration 7/100, Best fitness: -143471.31656365754
Iteration 8/100, Best fitness: -143471.31656365754
Iteration 9/100, Best fitness: -143471.31656365754
Iteration 10/100, Best fitness: -143471.31656365754
Iteration 11/100, Best fitness: -143471.31656365754
Iteration 12/100, Best fitness: -143471.31656365754
Iteration 13/100, Best fitness: -143471.31656365754
Iteration 14/100, Best fitness: -143471.31656365754


<ipython-input-11-cc029f57ba5f>:57: RuntimeWarning: invalid value encountered in subtract
  ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta


Iteration 15/100, Best fitness: -143471.31656365754
Iteration 16/100, Best fitness: -143471.31656365754
Iteration 17/100, Best fitness: -143471.31656365754
Iteration 18/100, Best fitness: -143471.31656365754
Iteration 19/100, Best fitness: -143471.31656365754
Iteration 20/100, Best fitness: -143471.31656365754
Iteration 21/100, Best fitness: -143471.31656365754
Iteration 22/100, Best fitness: -143471.31656365754
Iteration 23/100, Best fitness: -143471.31656365754
Iteration 24/100, Best fitness: -143471.31656365754
Iteration 25/100, Best fitness: -143471.31656365754
Iteration 26/100, Best fitness: -143471.31656365754
Iteration 27/100, Best fitness: -143471.31656365754
Iteration 28/100, Best fitness: -143471.31656365754
Iteration 29/100, Best fitness: -143471.31656365754
Iteration 30/100, Best fitness: -143471.31656365754
Iteration 31/100, Best fitness: -143471.31656365754
Iteration 32/100, Best fitness: -143471.31656365754
Iteration 33/100, Best fitness: -143471.31656365754
Iteration 34

### 5. Rolling element bearing

In [12]:
import numpy as np

# Constants for the problem
pi = np.pi

# Given constants
D = 160  # mm
d = 90   # mm
B_w = 30  # mm

# Define variable bounds
D_b_min = 0.15 * (D - d)  # 15% of (D - d)
D_b_max = 0.45 * (D - d)  # 45% of (D - d)
Z_min = 4
Z_max = 50

# Bounds for additional parameters
K_Dmin_min = 0.4
K_Dmin_max = 0.5
K_Dmax_min = 0.6
K_Dmax_max = 0.7
epsilon_min = 0.3
epsilon_max = 0.4
e_min = 0.02
e_max = 0.1
tau_min = 0.6
tau_max = 0.85

# Define bounds based on the existing constants
bounds = np.array([
    (D_b_min, D_b_max),  # D_b
    (K_Dmin_min, K_Dmin_max),  # K_Dmin
    (K_Dmax_min, K_Dmax_max),  # K_Dmax
    (Z_min, Z_max),      # Z
    (epsilon_min, epsilon_max),  # epsilon
    (e_min, e_max),      # e
    (tau_min, tau_max)   # tau
])

# Objective function
def objective(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    if D_b <= 25.4:
        C_d = f_c(D_b) * safe_power(Z, 2/3) * safe_power(D_b, 1.8)
    else:
        C_d = 3.647 * f_c(D_b) * safe_power(Z, 2/3) * safe_power(D_b, 1.4)
    return -C_d  # Minimize negative to maximize C_d

# Function to safely compute powers
def safe_power(base, exponent):
    if base < 0 and exponent % 2 == 0:  # Even exponent with a negative base
        return np.nan  # or return 0, depending on your needs
    elif base < 0:
        return np.nan  # or return a default value
    elif base == 0 and exponent < 0:
        return np.nan  # or return a default value
    return base ** exponent

# Function to calculate f_c
def f_c(D_b):
    gamma = D_b * np.cos(np.radians(30)) / (D_b + 90)  # Example angle of 30 degrees
    f_i = 0.515  # Fixed value for this example
    f_o = 0.515  # Fixed value for this example

    # Check if gamma is in the valid range
    if not (1 + gamma > 0 and 1 - gamma > 0):
        return 0  # Or set to a suitable fallback value

    # Calculate f_c value while ensuring gamma is within a valid range
    f_c_value = 37.91 * (1 + (1.04 * ((1 - gamma) / (1 + gamma))**1.72 * (f_i * (2 * f_o - 1) / (f_o * (2 * f_i - 1)))**0.41)**(10/3))**(-0.3)

    # Check if f_c_value is NaN
    if np.isnan(f_c_value):
        return 0  # Set to a fallback value if invalid

    return f_c_value

# Constraint functions
def constraint1(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return (2 * np.arcsin(D_b / (D - d)) / (D_b / (D - d))) - Z + 1  # g1 >= 0

def constraint2(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return 2 * D_b - K_Dmin * (D - d)  # g2 >= 0

def constraint3(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return K_Dmax * (D - d) - 2 * D_b  # g3 >= 0

def constraint4(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return -(tau * B_w - D_b)  # g4 <= 0

def constraint5(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return (0.5 * (D + d)) - (D - d)  # g5 >= 0

def constraint6(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return (0.5 + e) * (D + d) - (D - d)  # g6 >= 0

def constraint7(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return 0.5 * (D - (D - d) - D_b) - epsilon * D_b  # g7 >= 0

def constraint8(x):
    f_i = x[4]  # f_i
    return f_i - 0.515  # g8 >= 0

def constraint9(x):
    f_o = x[5]  # f_o
    return f_o - 0.515  # g9 >= 0

# Combine constraints
def evaluate_constraints(x):
    constraints = [
        constraint1,
        constraint2,
        constraint3,
        constraint4,
        constraint5,
        constraint6,
        constraint7,
        constraint8,
        constraint9
    ]
    return np.array([con(x) for con in constraints])

# Sunflower using above Objective Function

In [24]:
import numpy as np
import pandas as pd

def hybrid_optimization(fitness_function, constraints, num_plants, bounds, MaxDays, m, P, λ):
    """
    Hybrid Optimization Algorithm for the given problem.

    Parameters:
        fitness_function (callable): Objective function to minimize.
        constraints (list): List of constraint functions.
        num_plants (int): Number of plants (population size).
        bounds (np.ndarray): Variable bounds as a 2D array [[min1, max1], [min2, max2], ...].
        MaxDays (int): Maximum number of iterations.
        m (float): Percentage of worst plants to replace.
        P (float): Heat constant.
        λ (float): Step size scaling factor.

    Returns:
        tuple: Best solution found, its fitness value, and fitness evolution.
    """
    num_dimensions = bounds.shape[0]
    Xmin, Xmax = bounds[:, 0], bounds[:, 1]
    loss_values = []

    # Parameters
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability
    eta = np.random.rand()  # Weightage

    # Initialize population
    population = np.random.uniform(Xmin, Xmax, (num_plants, num_dimensions))

    # Define penalized fitness function
    def penalized_fitness(individual):
        penalties = sum(max(0, -con(individual)) for con in constraints)
        return fitness_function(individual) + 1e6 * penalties

    # Evaluate initial fitness
    fitness = np.array([penalized_fitness(ind) for ind in population])
    fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

    # Find the best solution
    sun_index = np.argmin(fitness)
    sun = population[sun_index]
    best_fitness = fitness[sun_index]
    loss_values.append(best_fitness)

    # Main loop
    for day in range(MaxDays):
        # Calculate direction vectors
        distances = np.linalg.norm(sun - population, axis=1)
        distances[distances == 0] = np.finfo(float).eps
        direction_vectors = (sun - population) / distances[:, np.newaxis]

        # Heat calculation
        heat = P / (4 * np.pi * distances**2)

        # Update population
        for i in range(num_plants):
            r = np.random.rand()
            step_size = λ * np.random.rand() * np.linalg.norm(Xmax - Xmin) / (2 * num_plants)
            if r < p:
                population[i] += ((np.random.rand() ** 2) * sun - population[i]) * fragrances[i] * eta + \
                                  (1 - eta) * step_size * direction_vectors[i]
            else:
                j = np.random.randint(0, num_plants)
                k = np.random.randint(0, num_plants)
                population[i] += ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta + \
                                  (1 - eta) * step_size * direction_vectors[i]

        # Keep population within bounds
        population = np.clip(population, Xmin, Xmax)

        # Evaluate new fitness
        fitness = np.array([penalized_fitness(ind) for ind in population])
        fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

        # Replace worst plants
        b = int(m / 100.0 * num_plants)
        worst_indices = np.argsort(fitness)[-b:]
        best_indices = np.argsort(fitness)[:b]
        best_plants = population[best_indices]

        for index, best_index in zip(worst_indices, best_indices):
            new_solution = (population[best_index] + population[best_index - 1]) / 2
            population[index] = np.clip(new_solution, Xmin, Xmax)

        # Update the best solution (sun)
        new_sun_index = np.argmin(fitness)
        if fitness[new_sun_index] < best_fitness:
            sun_index = new_sun_index
            sun = population[sun_index]
            best_fitness = fitness[new_sun_index]

        loss_values.append(best_fitness)
        a = np.random.rand()
        eta = np.random.rand()

        print(f"Day {day + 1}/{MaxDays}, Best fitness: {best_fitness}")

    return sun, best_fitness, loss_values


# Problem definition
problem = {
    "objective": objective,
    "constraints": [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7, constraint8],
    "bounds": bounds,
}

# Run the algorithm
num_plants = 30
MaxDays = 100
m = 20
P = 1.0
λ = 0.5

best_solution, best_fitness, fitness_evolution = hybrid_optimization(
    problem["objective"],
    problem["constraints"],
    num_plants,
    problem["bounds"],
    MaxDays,
    m,
    P,
    λ,
)

print("Best solution:", best_solution)
print("Best fitness:", best_fitness)

# ... (previous code) ...

# Save results to CSV
output_file = "optimization_results.csv"  # Change file extension to .csv

# Create a DataFrame for fitness evolution
results_df = pd.DataFrame({
    "Day": list(range(1, len(fitness_evolution) + 1)),
    "Fitness Evolution": fitness_evolution
})

# Create a DataFrame for best solution and fitness
best_solution_df = pd.DataFrame({
    "Variable": [f"x{i+1}" for i in range(len(best_solution))],
    "Value": best_solution
})
best_solution_df.loc[len(best_solution_df)] = ["Best Fitness", best_fitness]

# Save to CSV files
results_df.to_csv(output_file, index=False)  # Save fitness evolution to CSV
best_solution_df.to_csv("best_solution.csv", index=False)  # Save best solution to a separate CSV

print(f"Fitness evolution saved to {output_file}")
print("Best solution saved to best_solution.csv")

Day 1/100, Best fitness: 1343041.1787128765
Day 2/100, Best fitness: 1343041.1787128765
Day 3/100, Best fitness: 1343041.1787128765
Day 4/100, Best fitness: 1343041.1787128765
Day 5/100, Best fitness: 1343041.1787128765
Day 6/100, Best fitness: 1343041.1787128765
Day 7/100, Best fitness: 1343041.1787128765
Day 8/100, Best fitness: 1343041.1787128765
Day 9/100, Best fitness: 1343041.1787128765
Day 10/100, Best fitness: 1343041.1787128765
Day 11/100, Best fitness: 1343041.1787128765
Day 12/100, Best fitness: 1343041.1787128765
Day 13/100, Best fitness: 1343041.1787128765
Day 14/100, Best fitness: 1343041.1787128765
Day 15/100, Best fitness: 1343041.1787128765
Day 16/100, Best fitness: 1343041.1787128765
Day 17/100, Best fitness: 1343041.1787128765
Day 18/100, Best fitness: 1343041.1787128765
Day 19/100, Best fitness: 1343041.1787128765
Day 20/100, Best fitness: 1343041.1787128765
Day 21/100, Best fitness: 1343041.1787128765
Day 22/100, Best fitness: 1343041.1787128765
Day 23/100, Best fi

### 6. Belleville spring

In [25]:
import numpy as np

# Define constants for the problem
pi = np.pi
eps = 1e-8  # Small epsilon to avoid division by zero

# Given constants in SI units
P_max = 23912.4  # N
doh_max = 0.00508  # m
S = 1378950000  # Pa
E = 30e6  # Pa
mu = 0.3
H = 0.0508  # m
D_max = 0.305  # m

# Define variable bounds in SI units
D_e_min = 0.0254  # m (minimum outer diameter)
D_e_max = D_max  # m (maximum outer diameter)
D_i_min = 0.00254  # m (minimum inner diameter)
D_i_max = D_max  # m (maximum inner diameter)
t_min = 0.000254  # m (minimum thickness)
t_max = 0.0254  # m (maximum thickness)
h_min = 0.000254  # m (minimum height)
h_max = H  # m (maximum height)

# Define bounds based on the existing constants
bounds = np.array([
    (D_e_min, D_e_max),  # D_e
    (D_i_min, D_i_max),  # D_i
    (t_min, t_max),      # t
    (h_min, h_max)       # h
])

# Objective function
def objective(x):
    D_e, D_i, t, h = x
    return 0.07075 * pi * (D_e**2 - D_i**2) * t

# Constraint functions
def constraint1(x):
    D_e, D_i, t, h = x
    if D_i + eps <= 0:
        return np.inf  # Return high value if inner diameter is invalid
    K = D_e / (D_i + eps)  # Prevent division by zero
    if K <= 1:  # Check if K is valid for log calculation
        return np.inf  # Return high value for infeasibility

    alpha = (6 / pi * np.log(K + eps)) * ((K - 1) / (K + eps))**2
    beta = (6 / pi * np.log(K + eps)) * (((K - 1) / (np.log(K + eps))) - 1)
    gamma = (6 / pi * np.log(K + eps)) * ((K - 1) / 2)
    return S - (4 * E * doh_max / ((1 - mu**2) * alpha * D_e**2)) * (beta * (h - (doh_max / 2)) + gamma * t)  # g1 >= 0

def constraint2(x):
    D_e, D_i, t, h = x
    if D_i + eps <= 0:
        return np.inf  # Return high value if inner diameter is invalid
    K = D_e / (D_i + eps)  # Prevent division by zero
    if K <= 1:  # Check if K is valid for log calculation
        return np.inf  # Return high value for infeasibility

    alpha = (6 / pi * np.log(K + eps)) * ((K - 1) / (K + eps))**2
    return (4 * E * doh_max / ((1 - mu**2) * alpha * D_e**2)) * ((h - (doh_max / 2)) * (h - doh_max) * t + t**3) - P_max  # g2 >= 0

def constraint3(x):
    D_e, D_i, t, h = x
    a = h / (t + eps)  # Prevent division by zero
    return objective(x) * a - doh_max  # g3 >= 0

def constraint4(x):
    D_e, D_i, t, h = x
    return H - h - t  # g4 >= 0

def constraint5(x):
    D_e, D_i, t, h = x
    return D_max - D_e  # g5 >= 0

def constraint6(x):
    D_e, D_i, t, h = x
    return D_e - D_i  # g6 >= 0

def constraint7(x):
    D_e, D_i, t, h = x
    return 0.3 - h / ((D_e - D_i) + eps)  # g7 >= 0

# Combine constraints
def evaluate_constraints(x):
    return np.array([
        constraint1(x),
        constraint2(x),
        constraint3(x),
        constraint4(x),
        constraint5(x),
        constraint6(x),
        constraint7(x)
    ])

# Sunflower using above Objective Function

In [27]:
import numpy as np
import pandas as pd

def hybrid_optimization(fitness_function, constraints, num_plants, bounds, MaxDays, m, P, λ):
    """
    Hybrid Optimization Algorithm for the given problem.

    Parameters:
        fitness_function (callable): Objective function to minimize.
        constraints (list): List of constraint functions.
        num_plants (int): Number of plants (population size).
        bounds (np.ndarray): Variable bounds as a 2D array [[min1, max1], [min2, max2], ...].
        MaxDays (int): Maximum number of iterations.
        m (float): Percentage of worst plants to replace.
        P (float): Heat constant.
        λ (float): Step size scaling factor.

    Returns:
        tuple: Best solution found, its fitness value, and fitness evolution.
    """
    num_dimensions = bounds.shape[0]
    Xmin, Xmax = bounds[:, 0], bounds[:, 1]
    loss_values = []

    # Parameters
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability
    eta = np.random.rand()  # Weightage

    # Initialize population
    population = np.random.uniform(Xmin, Xmax, (num_plants, num_dimensions))

    # Define penalized fitness function
    def penalized_fitness(individual):
        penalties = sum(max(0, -con(individual)) for con in constraints)
        return fitness_function(individual) + 1e6 * penalties

    # Evaluate initial fitness
    fitness = np.array([penalized_fitness(ind) for ind in population])
    fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

    # Find the best solution
    sun_index = np.argmin(fitness)
    sun = population[sun_index]
    best_fitness = fitness[sun_index]
    loss_values.append(best_fitness)

    # Main loop
    for day in range(MaxDays):
        # Calculate direction vectors
        distances = np.linalg.norm(sun - population, axis=1)
        distances[distances == 0] = np.finfo(float).eps
        direction_vectors = (sun - population) / distances[:, np.newaxis]

        # Heat calculation
        heat = P / (4 * np.pi * distances**2)

        # Update population
        for i in range(num_plants):
            r = np.random.rand()
            step_size = λ * np.random.rand() * np.linalg.norm(Xmax - Xmin) / (2 * num_plants)
            if r < p:
                population[i] += ((np.random.rand() ** 2) * sun - population[i]) * fragrances[i] * eta + \
                                  (1 - eta) * step_size * direction_vectors[i]
            else:
                j = np.random.randint(0, num_plants)
                k = np.random.randint(0, num_plants)
                population[i] += ((np.random.rand() ** 2) * population[j] - population[k]) * fragrances[i] * eta + \
                                  (1 - eta) * step_size * direction_vectors[i]

        # Keep population within bounds
        population = np.clip(population, Xmin, Xmax)

        # Evaluate new fitness
        fitness = np.array([penalized_fitness(ind) for ind in population])
        fragrances = a * (np.sign(fitness) * (np.abs(fitness)) ** c) + np.finfo(float).eps

        # Replace worst plants
        b = int(m / 100.0 * num_plants)
        worst_indices = np.argsort(fitness)[-b:]
        best_indices = np.argsort(fitness)[:b]
        best_plants = population[best_indices]

        for index, best_index in zip(worst_indices, best_indices):
            new_solution = (population[best_index] + population[best_index - 1]) / 2
            population[index] = np.clip(new_solution, Xmin, Xmax)

        # Update the best solution (sun)
        new_sun_index = np.argmin(fitness)
        if fitness[new_sun_index] < best_fitness:
            sun_index = new_sun_index
            sun = population[sun_index]
            best_fitness = fitness[new_sun_index]

        loss_values.append(best_fitness)
        a = np.random.rand()
        eta = np.random.rand()

        print(f"Day {day + 1}/{MaxDays}, Best fitness: {best_fitness}")

    return sun, best_fitness, loss_values


# Problem definition
problem = {
    "objective": objective,
    "constraints": [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7],
    "bounds": bounds,
}

# Run the algorithm
num_plants = 30
MaxDays = 100
m = 20
P = 1.0
λ = 0.5

best_solution, best_fitness, fitness_evolution = hybrid_optimization(
    problem["objective"],
    problem["constraints"],
    num_plants,
    problem["bounds"],
    MaxDays,
    m,
    P,
    λ,
)

print("Best solution:", best_solution)
print("Best fitness:", best_fitness)

# ... (previous code) ...

# Save results to CSV
output_file = "optimization_results.csv"  # Change file extension to .csv

# Create a DataFrame for fitness evolution
results_df = pd.DataFrame({
    "Day": list(range(1, len(fitness_evolution) + 1)),
    "Fitness Evolution": fitness_evolution
})

# Create a DataFrame for best solution and fitness
best_solution_df = pd.DataFrame({
    "Variable": [f"x{i+1}" for i in range(len(best_solution))],
    "Value": best_solution
})
best_solution_df.loc[len(best_solution_df)] = ["Best Fitness", best_fitness]

# Save to CSV files
results_df.to_csv(output_file, index=False)  # Save fitness evolution to CSV
best_solution_df.to_csv("best_solution.csv", index=False)  # Save best solution to a separate CSV

print(f"Fitness evolution saved to {output_file}")
print("Best solution saved to best_solution.csv")

Day 1/100, Best fitness: 5833.2795486672185
Day 2/100, Best fitness: 5829.278977939505
Day 3/100, Best fitness: 5829.278977939505
Day 4/100, Best fitness: 5829.278977939505
Day 5/100, Best fitness: 5692.666056080287
Day 6/100, Best fitness: 5662.7949594149495
Day 7/100, Best fitness: 5659.001547892327
Day 8/100, Best fitness: 5555.157019656047
Day 9/100, Best fitness: 5356.815114233071
Day 10/100, Best fitness: 5320.354438121238
Day 11/100, Best fitness: 5272.80439507208
Day 12/100, Best fitness: 5244.558744971555
Day 13/100, Best fitness: 5242.941968764843
Day 14/100, Best fitness: 5124.739664866655
Day 15/100, Best fitness: 5124.739664866655
Day 16/100, Best fitness: 5111.649585201343
Day 17/100, Best fitness: 5111.649585201343
Day 18/100, Best fitness: 5111.649585201343
Day 19/100, Best fitness: 5111.649585201343
Day 20/100, Best fitness: 5111.649585201343
Day 21/100, Best fitness: 5111.649585201343
Day 22/100, Best fitness: 5111.649585201343
Day 23/100, Best fitness: 5111.649585201

In [ ]:
fitness_history.to_csv('PSO.csv', index=False)

NameError: name 'fitness_history' is not defined